# (1) Twitter Data
## (1.1) Getting Twitter data 2021 from API

In [3]:
import pandas as pd
from path import Path
from twarc import Twarc2, expansions
import json
import datetime
import os

In [4]:
from config import bearer_token

In [ ]:
client = Twarc2(bearer_token=bearer_token)

In [ ]:
user = 'elonmusk'
posts_dict = {
    'date':[],
    'text':[],
    'like_count':[],
    'reply_count':[],
    'retweet_count':[]
}

In [ ]:
# pull posts from Twitter and create a dictionary
user_timeline = client.timeline(user=user, exclude_replies=True, start_time=datetime.datetime(2021,1,1, 0, 0, 0) )
for page in user_timeline:
    result = expansions.flatten(page)
    for tweet in result:
        posts_dict['date'].append(tweet['created_at'])
        posts_dict['text'].append(tweet['text'])
        posts_dict['like_count'].append(tweet['public_metrics']['like_count'])
        posts_dict['reply_count'].append(tweet['public_metrics']['reply_count'])
        posts_dict['retweet_count'].append(tweet['public_metrics']['retweet_count'])

In [ ]:
# convert dictionary of posts to dataframe
twitter_2021 = pd.DataFrame.from_dict(posts_dict)
twitter_2021.head()

In [ ]:
# convert date to datetime datatype
twitter_2021['date'] = pd.to_datetime(twitter_2021['date']).dt.date.astype('datetime64')
twitter_2021.tail()

## (1.2) Getting Twitter data 2011 - 2020 from archive

In [ ]:
# load twitter data from csv file
file_to_load = os.path.join('Data', 'elon_musk_tweets_2011-2021.csv')
twitter_archive = pd.read_csv(file_to_load)
twitter_archive.head()

In [ ]:
# select and rename columns
twitter_archive_clean = twitter_archive[['date', 'tweet', 'nlikes', 'nreplies', 'nretweets']]\
                            .loc[(twitter_archive['reply_to'] == '[]') & (twitter_archive['retweet'] == False)]
twitter_archive_clean.columns=['date', 'text', 'like_count', 'reply_count', 'retweet_count']

# convert date to datetime datatype
twitter_archive_clean['date'] = pd.to_datetime(twitter_archive_clean['date']).dt.date.astype('datetime64')

# drop last row with 1 tweet in 2011
twitter_archive_clean.drop(twitter_archive_clean.tail(1).index,inplace=True)

twitter_archive_clean.head()

## (1.3) Clean the twitter data

In [ ]:
# concatenate 2 datasets to get tweets from 2011 to 2021
twitter_df_merged = pd.concat([twitter_2021, twitter_archive_clean])
twitter_df_merged.info()

In [ ]:
# Drop the NaNs
twitter_df_merged.dropna()

In [ ]:
# export all tweets for analysis in Tableau
twitter_df_merged.to_csv('Data/tweets_data_2011_2021_ungrouped.csv', index=False)

## (1.4) Preprocessing the Twitter data

**Preprocess the data by making it all lowercase. Remove a reasonable set of stopwords from the dataset and tokenize. Then, report the 10 most common words and their count. We need to iterate this process, adding some stop words as we understand the structure of the data. Justify additional stop words we've added.**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from datetime import datetime
from nltk.stem import PorterStemmer
import re
import nltk
nltk.download('stopwords')

In [ ]:
# group tweets that posted at the same day
def f(x):
     return pd.Series(dict(like_count = x['like_count'].sum(),
                        reply_count = x['reply_count'].sum(),
                        retweet_count = x['retweet_count'].sum(),
                        text = "{%s}" % ', '.join(x['text'])))

In [ ]:
twitter_df_merged = twitter_df_merged.groupby('date').apply(f).reset_index()
twitter_df_merged.head()


In [ ]:
twitter_df_merged.count()

In [ ]:
twitter_df = twitter_df_merged.copy()

In [ ]:
# Data Pre-processing and make the tweets all lowercase and remove stopwords.
# lower the tweets
twitter_df['preprocessed_text'] = twitter_df['text'].str.lower()

# remove apostrophe from words and url
twitter_df['preprocessed_text'] = [re.sub("('[a-z]+)\s", " ", row) for row in twitter_df['preprocessed_text']]
twitter_df['preprocessed_text'] = [re.sub("(')\s", " ", row) for row in twitter_df['preprocessed_text']]
twitter_df['preprocessed_text'] = [re.sub("(?:https:\/\/\S+)\s", "", row) for row in twitter_df['preprocessed_text']]

                                      
# filter out rest URLs
url_re = '(?:https?:\/\/)?(?:[^?\/\s]+[?\/])(.*)'
twitter_df['preprocessed_text'] = twitter_df['preprocessed_text'].apply(lambda row: ' '.join([word for word in row.split() if (not re.match(url_re, word))]))

# tokenize the tweets
tokenizer = RegexpTokenizer('[a-zA-Z]\w+\'?\w*')
twitter_df['tokenized_text'] = twitter_df['preprocessed_text'].apply(lambda row: tokenizer.tokenize(row))

#create an object of class PorterStemmer
porter = PorterStemmer()

# apply stemming
twitter_df['preprocessed_text'] = [porter.stem(row) for row in twitter_df['preprocessed_text']]   

# filter out stop words
en_stop_words = nltk.corpus.stopwords.words('english')
additional_stop_words =['amp', 'rt', 'th','co', 're', 've', 'kim', 'daca', 'us', 'it', 'th', 'you', 'haha', 'st', 'et', 'so', 'iii', 'also', 've', 'la', 're', 'the', 'https', 'wow', 'actually', 'due', 'ft', 'pcr', 'via', 'am', 'gt', 'com', 'since', 'in', 'me', 'and', 'btw', 'yesterday', 'ii', 'inu', 'on', 'http', 'to', 'vs', 'rd', 'ur', 'of', 'bs', 'km', 'est', 'em', 'lz', 'kms', 'aft', 'nd',  'here’s', 're', 'mqxfakpzf' 'mph', 'ht', 'etc', 'dm', 'doo']
en_stop_words.extend(additional_stop_words)

twitter_df['tokenized_text'] = twitter_df['tokenized_text'].apply(lambda row: [word for word in row if (not word in en_stop_words)])

df_tweets_clean = twitter_df.copy()
df_tweets_clean.head()

In [ ]:
df_tweets_clean = df_tweets_clean[['date', 'text', 'tokenized_text', 'like_count', 'reply_count', 'retweet_count']]
df_tweets_clean.head(10)

In [ ]:
# count unique words
def get_most_freq_words(str, n=None):
    vect = CountVectorizer().fit(str)
    bag_of_words = vect.transform(str)
    sum_words = bag_of_words.sum(axis=0) 
    freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
    freq =sorted(freq, key = lambda x: x[1], reverse=True)
    return freq[:n]
  
len(get_most_freq_words([ word for tweet in df_tweets_clean.tokenized_text for word in tweet]))

In [ ]:
df_tweets_clean.to_csv('data/tweets_data_2011_2021.csv', index=False)

## (1.5) Upload dataset to SQL Database

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine, inspect
import psycopg2
from config import user, password, hostname

In [ ]:
# Create engine
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{hostname}/twitter_vs_stocks')

# Use the Inspector to explore the database
inspector = inspect(engine)

In [ ]:
df_tweets_clean.to_sql('tweets_text', engine, if_exists ='replace',method='multi', index=False)